Projet de Données à la Connaissance
======

### Disclaimer
**NSFW : Nous ne cautionnons pas le vocabulaire ni le contenu de certains subreddits (sexuel, grossier, raciste, ou autres,...) **

## Introduction
Reddit est le 8ème site le plus populaire actuellement avec un milliard de visites par mois https://www.statista.com/statistics/443332/reddit-monthly-visitors/ . C'est un forum, organisé en sous-forums thématiques, appelés SubReddits.
Le but de notre projet est de comparer deux méthodes de clustering sur un dataset de posts et de commentaires sur Reddit. Plus précisement, nous souhaitons créer des clusters de SubReddit similaires, en prenant en compte les utilisateurs ou le texte.

## Données

### Origine
dataset : Les posts et commentaires écrits sur Reddit en février 2017
Word2Vec déjà entrainé par Google : https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit sur 100 milliards d'articles.

### Prétraitement
Les fichiers initiaux consistaient en une liste de JSON, chacun représentant un post ou un commentaire. Pour le seul mois de février 2017, cela représentait un total de plus de 55Go de données.

A partir des données de base, nous avons éliminé un grand nombre d'attribut, pour ne garder que le nom du SubReddit, les pseudos des utilisateurs, et le texte des posts/commentaires/titres. 

Nous n'avons pas gardé les activités des utilisateurs bannis.

Nous avons enregistré ces données pré-traitées avec le format CSV.

## Approche par texte
On souhaite analyser le contenu textuel de plusieurs SubReddits pour construire des clusters. Pour chaque SubReddit, le texte analysé provient des titres des posts, des posts eux-mêmes ainsi que des commentaires. 

Nous avons tokenisé, filtré, puis vectorisé des textes avec word2vec de Google. Nous aurions pu utiliser la même approche que par utilisateur, soit par sac de mots. Pour vectoriser le texte d'un SubReddit, nous moyennons les vecteurs de chaque mot. Ces vecteurs spécifiques à chaque SubReddit sont ainsi comparés par notre algorithme de clustering.

Le détail de cette approche est commenté plus bas dans le code.

## Approche par utilisateur
On souhaite analyser les utilisateurs communs sur plusieurs SubReddits pour construire des clusters.

Nous partons de l'hypothèse qu'un utilisateur s'intéressera souvent à plusieurs SubReddits de thématiques proches. Par exemple, un amateur de basket commentera sur un SubReddit des San Antonio Spurs et Golden State Warriors.

Le détail de cette approche est commenté plus bas dans le code.


## Choix de l'algorithme de clustering
Nous avons choisi l'algorithme présent sur Scikit-Learn, KMeans. Il a l'avantage d'être simple, et adapté à notre taille de problème. De plus, le seul paramètre est le nombre de clusters, ce qui permet un bonne comparaison entre nos deux approches. En effet, d'autres algortihmes tels que DBSCAN demandent une distance en paramètre, qui serait très différente entre les deux approches, et qui ne permettrait donc pas une bonne comparaison.


## Note
Dans la suite, vous pourrez faire tourner notre code. Nous vous présentons un exemple de clustering pour chaque approche dans les fichiers approche1clusters.txt et approche2clusters.txt. 

In [4]:
import sys
import math
import sys

import pandas as pd # Ouverture des données
import numpy as np # Calcul numérique
from sklearn.cluster import KMeans
from scipy.sparse import dok_matrix, csr_matrix
from sklearn.decomposition import TruncatedSVD

In [2]:
def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0

# Première approche : Clustering des SubReddits par NLP

In [3]:
import nltk # Traitement du texte
nltk.download('punkt')
import gensim # Pour lire word2vec
import re # Regex

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mathieu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


C:\Users\Mathieu\Anaconda3\envs\deeplearning\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
# Ouverture des données - posts
df_posts=pd.read_csv("data/reddit_posts.csv",sep=";", names=['Id','UserName','SubReddit','Title','Body'], header=None)
df_posts=df_posts.drop(['Id','UserName'], axis=1)
df_posts.SubReddit=df_posts.SubReddit.astype('category') # Pour réduire de 90% les besoins de mémoire sur cette series

# Rassemeblement de tout le text dans un series
df_posts['Text']=df_posts.Title.str.cat(df_posts.Body, sep=" ", na_rep="")
df_posts = df_posts.drop(['Body','Title'], axis=1)

print(df_posts.head())
print(f"Posts shape : {df_posts.shape}, taille en mémoire : {convert_bytes(sys.getsizeof(df_posts))}")

   SubReddit                                               Text
0      Honda                                    The new S2000? 
1   Patriots     I made a video for SB LI supporting our Pats! 
2  AskReddit  Trump supporters - Do you feel that he is doin...
3  doctorwho                        The 13th Doctor we deserve 
4    pystats                       Introduction to Correlation 
Posts shape : (4468114, 2), taille en mémoire : 690.8 MB


In [4]:
# Ouverture des données - Commentaires
df_comments=pd.read_csv("data/reddit_comments.csv",sep=";", names=['Id','UserName','SubReddit','Text'], header=None)
df_comments=df_comments.drop(['Id','UserName'], axis=1)
df_comments.SubReddit=df_comments.SubReddit.astype('category')
print(df_comments.head())
print(f"Comments shape : {df_comments.shape}, taille en mémoire : {convert_bytes(sys.getsizeof(df_comments))}")

             SubReddit                                               Text
0        pokemontrades                        Regigigas and mega alakazam
1                  Amd                     I think we deserve more time! 
2  KitchenConfidential  You looking for a gig? In Oak cliff may need s...
3          LiverpoolFC               Just asking but being a dick helped 
4      ecigclassifieds              I am interested. What is your paypal?
Comments shape : (35561917, 2), taille en mémoire : 5.3 GB


In [5]:
# Fusion des deux dataframes
df_dataset=pd.concat([df_posts, df_comments])
del df_posts, df_comments # On libère la mémoire
print(df_dataset.head())
print(f"Full Dataset shape : {df_dataset.shape}, taille en mémoire : {convert_bytes(sys.getsizeof(df_dataset))}")


             SubReddit                                               Text
0        pokemontrades                        Regigigas and mega alakazam
1                  Amd                     I think we deserve more time! 
2  KitchenConfidential  You looking for a gig? In Oak cliff may need s...
3          LiverpoolFC               Just asking but being a dick helped 
4      ecigclassifieds              I am interested. What is your paypal?
Comments shape : (40030031, 2), taille en mémoire : 8.6 GB


In [9]:
# Tokenization des textes
df_dataset.Text=df_dataset.Text.fillna("")
df_dataset.Text=df_dataset.Text.str.replace(r"http\S+", "")
df_dataset.Text=df_dataset.Text.str.replace(r"\[.*\]","")
df_dataset.Text=df_dataset.Text.str.replace("NaN","")
df_dataset.Text=df_dataset.Text.str.replace(r"@\S+", "")
df_dataset.Text=df_dataset.Text.str.replace(r"\W+", " ")
df_dataset.Text=df_dataset.Text.str.lower()

In [10]:
print(df_dataset.head())
print(f"Taille en mémoire de df_dataset: {convert_bytes(sys.getsizeof(df_dataset))}")

   SubReddit                                               Text
0      Honda                                     the new s2000 
1   Patriots      i made a video for sb li supporting our pats 
2  AskReddit  trump supporters do you feel that he is doing ...
3  doctorwho                        the 13th doctor we deserve 
4    pystats                       introduction to correlation 
Taille en mémoire de df_posts: 8.1 GB


In [13]:
# Filtrage à partir des stop words
stop_words = set(nltk.corpus.stopwords.words('english'))
df_dataset.Text=df_dataset.Text.apply(lambda l: " ".join([w for w in l.split(" ") if w not in stop_words]))

In [37]:
print(df_dataset.head())
print(f"Taille en mémoire de df_dataset: {convert_bytes(sys.getsizeof(df_dataset))}")

   SubReddit                                               Text
0      Honda                                         new s2000 
1   Patriots                  made video sb li supporting pats 
2  AskReddit  trump supporters feel good job far im trying s...
3  doctorwho                               13th doctor deserve 
4    pystats                          introduction correlation 
Taille en mémoire de df_dataset: 7.1 GB


In [4]:
# Enregistrement du dataset
df_dataset.to_csv("df_dataset.csv")
#df_dataset=pd.read_csv("df_dataset.csv")
#df_dataset= df_dataset.drop(['Unnamed: 0'], axis=1)
#df_dataset.Text=df_dataset.Text.apply(lambda x: "" if pd.isnull(x) else x)
#df_dataset.head()

,SubReddit,Text
0,Honda,new s2000
1,Patriots,made video sb li supporting pats
2,AskReddit,trump supporters feel good job far im trying s...
3,doctorwho,13th doctor deserve
4,pystats,introduction correlation


In [5]:
# Regroupement du texte pour chaque SubReddit
grouped_dataset=df_dataset.groupby(['SubReddit'])['Text'].apply(lambda x : " ".join(x))
del df_dataset
print(grouped_dataset.head())
print(grouped_dataset.shape)
print(f"Taille en mémoire de df_dataset: {convert_bytes(sys.getsizeof(grouped_dataset))}")

SubReddit
0000000027544444                              nothing see move along 
000014              feed stock youre zalo apk download free commun...
000016                                                               
000webhost          welcome 000webhost subreddit subreddit moderat...
001                              latina girl gives guy lap dance  001
Name: Text, dtype: object
(94288,)
Taille en mémoire de df_dataset: 2.5 GB


In [6]:
EMBEDDING_DIM = 300
word2vec = gensim.models.KeyedVectors.load_word2vec_format("data/GoogleNews-vectors-negative300.bin.gz", binary=True)

In [24]:
# Vectorisation d'un texte

def aggregate_word2vec(words):
    """
    Lit une séquence de mots, et renvoie la moyenne des vecteurs associés à chaque mots avec word2vec de la séquence de mots
    words : string
    return : vec
    """
    vec=np.zeros(shape=(EMBEDDING_DIM,))
    n_sum=0
    for word in words.split(" "):
        if word in word2vec:
            n_sum+=1
            vec=np.add(vec, word2vec[word])
    if n_sum:
        return vec/n_sum
    else:
        vec

In [17]:
# On crée notre tenseur d'entrée pour l'algorithme de Clustering

n_subreddit = grouped_dataset.shape[0]
subreddit_id={}

subreddit_vectorized =np.zeros(shape=(n_subreddit, EMBEDDING_DIM,)) # Tenseur d'entrée

sub_id=0
for index,item in grouped_dataset.iteritems():
    subreddit =index
    words=item
    subreddit_vectorized[sub_id, :]=aggregate_word2vec(words)
    subreddit_id[subreddit]=sub_id
    if sub_id%100==0:
        print(f"Etape : {sub_id}/{n_subreddit}")
    sub_id+=1

Etape 0/94288
Etape 100/94288
Etape 200/94288
Etape 300/94288
Etape 400/94288
Etape 500/94288
Etape 600/94288
Etape 700/94288
Etape 800/94288
Etape 900/94288
Etape 1000/94288
Etape 1100/94288
Etape 1200/94288
Etape 1300/94288
Etape 1400/94288
Etape 1500/94288
Etape 1600/94288
Etape 1700/94288
Etape 1800/94288
Etape 1900/94288
Etape 2000/94288
Etape 2100/94288
Etape 2200/94288
Etape 2300/94288
Etape 2400/94288
Etape 2500/94288
Etape 2600/94288
Etape 2700/94288
Etape 2800/94288
Etape 2900/94288
Etape 3000/94288
Etape 3100/94288
Etape 3200/94288
Etape 3300/94288
Etape 3400/94288
Etape 3500/94288
Etape 3600/94288
Etape 3700/94288
Etape 3800/94288
Etape 3900/94288
Etape 4000/94288
Etape 4100/94288
Etape 4200/94288
Etape 4300/94288
Etape 4400/94288
Etape 4500/94288
Etape 4600/94288
Etape 4700/94288
Etape 4800/94288
Etape 4900/94288
Etape 5000/94288
Etape 5100/94288
Etape 5200/94288
Etape 5300/94288
Etape 5400/94288
Etape 5500/94288
Etape 5600/94288
Etape 5700/94288
Etape 5800/94288
Etape 590

Etape 46200/94288
Etape 46300/94288
Etape 46400/94288
Etape 46500/94288
Etape 46600/94288
Etape 46700/94288
Etape 46800/94288
Etape 46900/94288
Etape 47000/94288
Etape 47100/94288
Etape 47200/94288
Etape 47300/94288
Etape 47400/94288
Etape 47500/94288
Etape 47600/94288
Etape 47700/94288
Etape 47800/94288
Etape 47900/94288
Etape 48000/94288
Etape 48100/94288
Etape 48200/94288
Etape 48300/94288
Etape 48400/94288
Etape 48500/94288
Etape 48600/94288
Etape 48700/94288
Etape 48800/94288
Etape 48900/94288
Etape 49000/94288
Etape 49100/94288
Etape 49200/94288
Etape 49300/94288
Etape 49400/94288
Etape 49500/94288
Etape 49600/94288
Etape 49700/94288
Etape 49800/94288
Etape 49900/94288
Etape 50000/94288
Etape 50100/94288
Etape 50200/94288
Etape 50300/94288
Etape 50400/94288
Etape 50500/94288
Etape 50600/94288
Etape 50700/94288
Etape 50800/94288
Etape 50900/94288
Etape 51000/94288
Etape 51100/94288
Etape 51200/94288
Etape 51300/94288
Etape 51400/94288
Etape 51500/94288
Etape 51600/94288
Etape 5170

Etape 91800/94288
Etape 91900/94288
Etape 92000/94288
Etape 92100/94288
Etape 92200/94288
Etape 92300/94288
Etape 92400/94288
Etape 92500/94288
Etape 92600/94288
Etape 92700/94288
Etape 92800/94288
Etape 92900/94288
Etape 93000/94288
Etape 93100/94288
Etape 93200/94288
Etape 93300/94288
Etape 93400/94288
Etape 93500/94288
Etape 93600/94288
Etape 93700/94288
Etape 93800/94288
Etape 93900/94288
Etape 94000/94288
Etape 94100/94288
Etape 94200/94288


In [19]:
# Enregistrement des données

#grouped_dataset.to_csv("grouped_dataset.csv")
#np.save("subreddit_vectorized.npy", subreddit_vectorized)

In [26]:
subreddit_vectorized=np.nan_to_num(subreddit_vectorized)

In [27]:
# Clustering
N_CLUSTERS = 1000
N_COMPONENTS = 300
db = KMeans(n_clusters=N_CLUSTERS).fit_predict(subreddit_vectorized)

In [28]:
# Affichage des résultats
for n in range(N_CLUSTERS):
    l = []
    for subreddit, sub_id in subreddit_id.items():
        cluster = db[sub_id]
        if cluster == n:
            l.append(subreddit)
    print("Cluster n°{}, size: {}".format(n, len(l)))
    print(l)
    print("\n")


Cluster n°0, size: 2
['drawsomething', 'feederwatchers']


Cluster n°1, size: 100
['4thwallconsumerism', 'Aerobatics', 'AirPorn', 'Airbus', 'Airline', 'AssCorn', 'Atrimus299', 'AvGeeks', 'AviationCanada', 'BestDrones', 'BismarckHQ', 'BlackLightning', 'BlackLightningCW', 'ChannelAbsoluteZero', 'DavidDrone', 'DroneApps', 'Drone_Reviews', 'DronesGoneWrong', 'ElijahFlabberghast', 'GeneralSurgery', 'Greer', 'Hamilton_watches', 'Haustinson', 'HelicoptersWithArms', 'HowNotToFlyaDrone', 'InvertedMasterRace', 'Jetpackfighter', 'LauncherOne', 'MemeAddictsAnonymous', 'Militarymachines', 'MorningWood', 'MultiGP', 'NTSB_database', 'NewsByReddit', 'NightGlider', 'OldSchoolTools', 'OrigamiAirEnforcer', 'PassengerDrones', 'PilotEpisodes', 'Planes', 'Planespotting', 'Quadrocopter', 'RCPlanes', 'RC_Helicopters', 'RadixEngine', 'RaptorAircraft', 'Redheart', 'RichBoy', 'Science_for_me', 'SnapSimulator', 'SuperMechs', 'TCMakerspace', 'TalesFromTheMilitary', 'Toys4Kids', 'UAS', 'VIAR', 'WatchPigeonsDie', 'Y

Cluster n°8, size: 5
['GraphicsOrNot', 'IfOrderThenGrow', 'LightBondage', 'TravelTR', 'khaosanhotel']


Cluster n°9, size: 106
['50PLUS', 'ADHS', 'AllesFaktenFUN', 'AmbientAssistedLiving', 'Arbeiterbewegung', 'Armstadt', 'Bares', 'BitcoinAT', 'Boerse', 'Braunschweig', 'BuchDesMonats', 'CastropRauxel', 'CodBlackOpsIII', 'ContraMagazin', 'DE_PM', 'DIE_LINKE', 'DSA_RPG', 'DasGuteLeben', 'DeutschlandRussland', 'Dorfen', 'Eltern', 'Erotik', 'FestundFlauschig', 'Freising', 'Fussball_eh', 'Gerontologie', 'Gittertiere', 'HalleSaale', 'HautundHaare', 'Helmstedt', 'Hoerbuecher', 'Ichfickedichdubastard', 'IrrsinnigeKnaben', 'IrsinnigeKnaben', 'Kassel', 'Kaufhausverwaltung', 'MBCafe', 'MBPlenarsaal', 'MBZeitung', 'MannausSachsen', 'Mechatroniker', 'Muenster', 'NRW', 'OfficialPapaplatte', 'P0sa', 'Parteiball', 'Pflege', 'Rollenspiele', 'Rostock', 'RusslandNachrichten', 'SDGermany', 'SPDde', 'SRGG', 'Schamanenstube', 'Schreibkunst', 'Soylent_DE', 'Spanisch', 'SpankingStories', 'Splittermond', 'Styli

Cluster n°23, size: 8
['As_it_is', 'ChicagoGuides', 'ChicagolandMusic', 'Landmarks', 'canadian', 'chicagoband', 'gaychicago', 'sonreal']


Cluster n°24, size: 5
['AdviseMovie', 'RMovies', 'YPCareerAdvice', 'getajobmovie', 'marefriends']


Cluster n°25, size: 1
['Dumps']


Cluster n°26, size: 150
['Alkmaar', 'Almere', 'Armoowasright', 'BELGICA', 'BitcoinNL', 'Bitterbal', 'BoerHarm', 'Brabant', 'Breda', 'Brinta', 'Brugge', 'CirkeltrekTweedracht', 'DNDNL', 'DePARTIJ', 'DeStagiair', 'De_Republiek', 'De_Stokvis', 'DenBosch', 'DocentVO', 'Duif', 'DutchFIRE', 'DutchProducers', 'Eredivisie', 'FHICT', 'FUTNederland', 'FamilieVTM', 'FijiStudios', 'FriendsOfTheMoai', 'Frikandel', 'Frisia', 'Geschiedenis', 'Glitterplaatjes', 'GoHGuildNL', 'GroenLinks', 'GroeneMiddag', 'Hedel', 'Heerlen', 'Hyves', 'Independer', 'Kybernetes', 'Landstede', 'Leeuwarden', 'Leiden', 'Lelystad', 'Leuven', 'MANSE', 'Mechelen', 'MediwietNieuws', 'NasheedSquad', 'NathanxAcid', 'NederlandseProblemen', 'Nehalennia', 'Netherla

Cluster n°39, size: 96
['90sTrees', '90sgifs', 'Aalborg', 'Aarhus', 'Aktiemarknaden', 'BUENZLI', 'Bira', 'Boras', 'Bron', 'CoolRunnings', 'DANMAG', 'DanishEnts', 'DutchComedy', 'EUPol', 'FEGLE', 'Fmsweden', 'GODenmark', 'Gothenburg', 'Gresham', 'HalloAusBerlin', 'HeatItUp', 'IPPJ', 'Iceland', 'Jonkoping', 'LGEG6', 'Liberstad', 'Malmoe', 'Melodifestivalen', 'Muzic', 'NFLse', 'Naturvidenskab', 'NifCast', 'NorskFotball', 'NorskHipHop', 'NorthernPathfinders', 'Nyttigbras', 'Polhem', 'Polhemskolan', 'RenaultEV', 'RoGonewild', 'Rudbeck', 'SEbold', 'Skidor', 'Slagelse', 'Sniffer1994', 'Sonadow', 'Spel', 'Spil', 'Sumsar', 'SvenskmedAspergers', 'Sverige', 'TwitchDanmark', 'achorsens', 'ankdammen', 'arkitektur', 'asatro', 'atter', 'babylon', 'bybanen', 'dankmark', 'danskegamere', 'danskrap', 'dkfinance', 'dkpol', 'downloadsoft', 'ernstiz', 'frekhaug', 'gadgeti', 'giffar', 'helsommemegmegs', 'jura', 'justfinishedreading', 'kundtjanst', 'lanparty2017', 'medialife', 'odense', 'oslo', 'paradiseDK', 

Cluster n°47, size: 213
['360RealtorsEvent', '4G63', 'AF_POC', 'AVdeals', 'AmazonEssentials', 'AmazonUnder25', 'Amazon_Dealz', 'ArtisanMicrowaves', 'Asmongold', 'BadPranks', 'Badgers', 'BangandOlufsen', 'BazaarMarkets', 'BeoPlay', 'BestDeal', 'BestOfEbay', 'BestOfOnlineDeals', 'BestShopping', 'BestThings', 'BuyVaporizers', 'CannaVape', 'CanonCamera', 'CaperDeals', 'CatProducts', 'Celebrityhips', 'ChattanoogaMarket', 'CheapDSLR', 'CheekiBreekism', 'Collar', 'ComputerSoftware', 'Corrections', 'CustomiPhoneCase', 'DIY_Kawaii', 'DIY_Science', 'DealsCouponsBargains', 'DealsReddit', 'DiscountedVape', 'Dogecoin_Business', 'Dronescenter', 'Eargasm', 'EbayWTF', 'Eurocom', 'Fifth_Harmony', 'FunnyStuff', 'Funny_Commercials', 'GTX1070', 'Gachapon', 'GearPorn', 'Gopherwatch', 'Guzheng', 'HackedGadgetsRss', 'HamFest', 'HauteHorlogerie', 'Hermes', 'Hero4', 'Hip_Flasks', 'Holograms', 'Home', 'Horology', 'HotGirlsInTankTops', 'IGJ', 'JohnDay', 'JustHenryVIIIThings', 'Kangertech', 'KateMoss', 'Koji_Kend

...



Cluster n°944, size: 1
['mutantyearzero']


Cluster n°945, size: 1
['im12andimadesigner']


Cluster n°946, size: 137
['AirCraftPorn', 'AmatuarYoutube', 'AntarcticaPics', 'Arianespace', 'Astronomy', 'AtomicPorn', 'AutonomousNews', 'AvorionShips', 'BFGArmada', 'BabyHighChairs', 'Bangor', 'COADE', 'Colonizemars', 'DistantWorlds', 'EliteBountyHunters', 'EliteLore', 'EliteMiners', 'FLOX', 'FPVplanes', 'FromTheDepths', 'FukushimaWebCamWatch', 'GrandTheftAutoVI', 'HaloFleetBattles', 'HeavySeas', 'Hellion', 'ISRO', 'ISS', 'ImaginaryStarships', 'InterstellarPilots', 'JustRowedIntoTheShop', 'KSPMemes', 'KerbalAcademy', 'KerbalSpaceProgram', 'LessCredibleDefence', 'MH370', 'MagnificentTrumpet', 'MilitaryConspiracy', 'MilitaryGfys', 'MilitaryPorn', 'MusicPlayer', 'Nautical', 'NoMansSkyShips', 'PaperAirplanes', 'RANK10TRAINS', 'Reassembly', 'RebelGalaxy', 'Remington', 'RocketLab', 'SEEDGundam', 'SFFGazer', 'SpaceGifs', 'SpaceLaunchSystem', 'SpaceLegend', 'SpaceXLounge', 'SpecialAccess', 'StarWars

Cluster n°962, size: 0
[]


Cluster n°963, size: 74
['52weeksofcooking', 'Abilene', 'ArtisanBread', 'BackyardChicken', 'BloomingtonNormal', 'Bradstion', 'BridgeSims', 'CarlsJr', 'CheerUpAncap', 'ChickFilA', 'CowboyAction', 'Dominos', 'DvilleGA', 'EndTipping', 'EventPlanners', 'ForestHills', 'FortCollinsNewInTown', 'Freefood', 'Fremont', 'FrugalTO', 'Gilroy', 'Hayward', 'IndyFood', 'MIONIX', 'MRE', 'Malaga', 'McDonalds', 'NKU', 'NYCGuides', 'NerdSluts', 'Newberg', 'PapaJohns', 'Popeyes', 'PortStLucie', 'RedRaiders', 'SarcasticThoughts', 'SpringBranch', 'SpringfieldIL', 'Staunton', 'Sudbury', 'Tooele', 'VEGKC', 'VeganPogrom', 'Watsonville', 'bemidji', 'bemidjistate', 'blono', 'blueapron', 'casualllama', 'chico', 'circlejerknyc', 'eaganmn', 'fonddulac', 'forestvillagegame', 'gurgaon', 'hamtramck', 'ladylambthebeekeeper', 'lifequotes', 'mediaquotes', 'middletown_ct', 'petsofthemasterrace', 'pizzahut', 'qdoba', 'sanantoniofood', 'sanmarcos', 'sheboygan', 'shitseanmauneysays', 'statecollege'

Cluster n°974, size: 173
['1000aday', '1200YearsAgo', 'ALJAZEERAauto', 'Actually_Deplorable', 'AddictedToBangGood', 'AgeOfEmpires3', 'AlJazeera', 'AntiCo', 'Antiglobalism', 'Armeniangenocide', 'AskIndianMen', 'AustPropertyPonzi', 'BasementDweller', 'BostonIndie', 'BreitbartNews', 'CAP_Canada', 'CommunismAnarchy', 'Congo', 'CottonField', 'CuriousSpace', 'DeepMoleIntelligence', 'Denbighshire', 'Dubaireddit', 'ERDOGAN_INSULTS', 'EastAsianTruth', 'EuropeLife', 'EuropeanPeoples', 'European_New_Right', 'Europer', 'Freedom', 'Freekashmir', 'GlobalEntertainment', 'GlobalTerrorism', 'ImmigrationReform', 'IndianLeft', 'Indianpolitics', 'InfidelTube', 'IslamUnveiled', 'Islamiyat', 'Islamization', 'IsraelWatch', 'Jallikattu', 'JordanianCivilWar', 'JustTheTopNews', 'LibyanCrisis', 'MYPakistan', 'MainstreamPropaganda', 'McCain', 'Melkite', 'MessageofIslam', 'MilitaryMartialArts', 'NATOrussianconflict', 'NYCSocialism', 'NationalAlliance', 'Nationalism', 'OnlyInDubai', 'PINAC', 'PiratesOnline', 'Polis

Cluster n°980, size: 71
['1channel', 'AztlanPeopleTwitter', 'BADESABAVideos', 'BetweenPenuhBerkah', 'CrankParachute', 'DogsPurpose9jefis', 'ElysianEmpire', 'FallenSunGaming', 'FencesJumatBerkah', 'FlightSimulatorDeals', 'FounderJumatBerkah', 'G2Play', 'GoTlinks', 'GoldPutlockerFREE', 'Grammyslive', 'HIDDENFIGURESForFree', 'HIUBJkmnkja', 'HiddenMingguFun', 'Himiwako', 'ImStreaming', 'Isha', 'KlausFiehe', 'Kondrik', 'Kuledud3', 'LionPutlockerFREE', 'LiveStreaming', 'Livestream', 'MSFC', 'MaddijayySlaves', 'Mjayyroxx_at_cod', 'Monsterni9uq', 'MoonlightPenuhBerkah', 'NegroPenuhBerkah', 'PatriotsDaysuay9su', 'PokemonGoLiverpool', 'RINGSForFree', 'ResidentJumatBerkah', 'SDFGHSANTRDFFGHYTRS', 'Siltti', 'SleeplessMingguFun', 'SplitForFree', 'Streamerism', 'SwC_Poker', 'TheDevision', 'TwitchTeens', 'UFC209Live', 'Veriax', 'WWE_SC', 'WarMusic', 'alo', 'azuritereaction', 'barbapapaclan', 'calexil', 'daytona17', 'daytonaspeed', 'ea_ufc', 'faulerhans', 'freecell', 'gamesbrasil', 'giraffe', 'holufcx

Cluster n°993, size: 126
['100sets', '4sentencegamereviews', 'Amateur_central', 'AskDumbQuestionsHere', 'BetterThanPorn', 'BitcoinBums', 'BlackGirlsLoveAnal', 'BlackGirlsLoveFacials', 'BrainwashedAmerica', 'CPST', 'ChineseReddit', 'CollegeGirlPorn', 'ComicReviews', 'Dating_a_widower', 'DeafGifs', 'FLBP', 'GayWrongs', 'GeneralNetworking', 'GirlsOfGaming', 'HORNY_GIRLS_PORN', 'HauntedRVs', 'HomeRemodelingToday', 'HotCollegeGirls', 'HudsonMusicProject', 'Indoctrinated', 'Kissimmee', 'KokomiShiina', 'LEGOreviews', 'LaughingSteam', 'MakeStories', 'Medwedia', 'MixedWrestlingNews', 'NSFW_SororitySluts', 'NSFW_TwinkVideos', 'OrgySex', 'PerrieEdwards', 'PokemonGoFortWayne', 'Pokemonfriendsafari', 'PornHardcoreTw', 'PrisonSex', 'PsychWardChronicles', 'Purism', 'REMath', 'RTgamers', 'RayHudsonQuotes', 'RepublicOfReddit', 'Rozetta', 'SexgirlxXx', 'ShittyAskFitness', 'ShittySeduction', 'SnapNudes', 'Sparity', 'StpaulsSchool', 'SwingerCouples', 'Synthetic_companions', 'TheCoverUp', 'TheFameGirls', '

# Deuxième approche : Clustering des SubReddits par Utilisateurs

In [5]:
# Ouverture des données - posts et commentaires
df_posts=pd.read_csv("data/reddit_posts.csv",sep=";", names=['Id','UserName','SubReddit','Title','Body'], header=None)
print("Posts loaded into dataframe")
df_comments=pd.read_csv("data/reddit_comments.csv",sep=";", names=['Id','UserName','SubReddit','Comment'], header=None)
print("Comments loaded into dataframe")

Posts loaded into dataframe
Comments loaded into dataframe


In [6]:
print(len(df_posts))
print(df_posts.head())
print(len(df_comments))
print(df_comments.head())

4468114
   Id        UserName  SubReddit  \
0   1           SRASC      Honda   
1   3         JRabone   Patriots   
2   4         adamorn  AskReddit   
3   5        JakeM917  doctorwho   
4  10  DataScienceInc    pystats   

                                               Title       Body  
0                                     The new S2000?        NaN  
1      I made a video for SB LI supporting our Pats!        NaN  
2  Trump supporters - Do you feel that he is doin...  [removed]  
3                         The 13th Doctor we deserve        NaN  
4                        Introduction to Correlation        NaN  
35561917
   Id        UserName            SubReddit  \
0   1        crash787        pokemontrades   
1   2    Cuntosaurous                  Amd   
2   3         dwmills  KitchenConfidential   
3   4  GodsGiftInBeef          LiverpoolFC   
4   5       Smeeks765      ecigclassifieds   

                                             Comment  
0                        Regigigas and

In [7]:
# Retrait des colonnes inutiles (Title et Body pour les posts, Comment pour les commentaires)
df_posts_dropped = df_posts.drop(labels=['Title', 'Body'], axis=1)
df_comments_dropped = df_comments.drop(labels='Comment', axis=1)

# Création de séries contenant les users distincts et les subreddits distincts
users = df_posts_dropped["UserName"].append(df_comments_dropped["UserName"])
users.drop_duplicates(inplace=True)
nb_users = len(users)

subreddits = df_posts_dropped["SubReddit"].append(df_comments_dropped["SubReddit"])
subreddits.drop_duplicates(inplace=True)
nb_subreddits = len(subreddits)

# Concaténation des posts et des commentaires
all_data = df_posts_dropped.append(df_comments_dropped)

print(len(df_posts_dropped), nb_users, nb_subreddits)

4468114 3330748 94288


In [8]:
# On ne garde que les utilisateurs ayant posté plus de 10 messages sur le mois
grouped_users = all_data.groupby("UserName")["Id"].count()
grouped_users_removed = list(grouped_users[grouped_users > 10].index)
print(grouped_users_removed[:50], len(grouped_users_removed))

# On ne garde que les subreddits ayant reçu plus de 10 messages sur le mois
grouped_subreddits = all_data.groupby("SubReddit")["Id"].count()
grouped_subreddits_removed = list(grouped_subreddits[grouped_subreddits > 10].index)
print(grouped_subreddits_removed[:50], len(grouped_subreddits_removed))

['----------_--------', '--------__--------', '--------cool--------', '-------_-----', '------________', '-----BroAway-----', '-----McSqeezie-----', '-----_------_---', '-----_____-----_____', '-----fuck-----', '-----iMartijn-----', '----MXE----', '----Nomad----', '----Val----', '----__-__----', '----_____----', '----josh', '----root', '---Blue---', '---CMFinley---', '---David---', '---E', '---Earth---', '---JustMe---', '---Kai---', '---Lucifer---', '---R', '---S-U-C-C---', '---TheFierceDeity---', '---YNWA---', '---_---_---_---7', '---_--__-_-----', '---_-___', '---fantastic---', '---lol', '---o0o---', '---sniff---', '--23---', '--444--', '--ACAB--', '--AJ--', '--ANUSTART-', '--ATCQ--', '--AXIOM--', '--Aeo--', '--Arrow--', '--BMO--', '--Beetlejuice--', '--BlueHat--', '--CAT--'] 695616
['01246', '07Scape', '098f6bcd4621d373', '0ad', '0x3642', '1000thworldproblems', '100DaysofKeto', '100in28', '100movies365days', '100pushups', '100sets', '100sexiest', '100yearclub', '100yearsago', '101Wi

In [9]:
# Initialisation d'une matrice creuse et de dictionnaires users --> id et subreddits --> id
users_id, subreddits_id = {}, {}
user_item_matrix = dok_matrix((len(grouped_users_removed), len(grouped_subreddits_removed)), dtype=int)

# On groupe les posts par username et subreddit (équivalent de l'opération GROUP BY en SQL), on compte le nombre d'occurences par groupe, 
# et on ne garde que ceux ayant un username et un subreddit dans les listes calculées dans la cellule ci-dessus
grouped_comments = all_data.groupby(["UserName","SubReddit"])["Id"].count().reset_index()
grouped_comments = grouped_comments.loc[(grouped_comments["UserName"].isin(grouped_users_removed)) & (grouped_comments["SubReddit"].isin(grouped_subreddits_removed))]

print(type(grouped_comments))
print(grouped_comments)
c = 0
# On itère sur les groupes pour contruire la matrice
for index,item in grouped_comments.iterrows():
    c += 1
    username = item["UserName"]
    subreddit = item["SubReddit"]
    number = item["Id"]
    try:
        i = users_id[username]
    except:
        i = len(users_id)
        users_id[username] = i
    try:
        j = subreddits_id[subreddit]
    except:
        j = len(subreddits_id)
        subreddits_id[subreddit] = j
    user_item_matrix[i, j] = int(math.ceil(math.log(1 + number)))
    if c%100000 == 0:
        print(c, sys.getsizeof(user_item_matrix), sys.getsizeof(subreddits_id))

print("Finished")

toto
<class 'pandas.core.frame.DataFrame'>
                      UserName             SubReddit  Id
13         ----------_--------       leagueoflegends   9
14         ----------_--------                   wow   3
20          --------__--------              Winnipeg  72
21        --------cool--------               Fitness  10
22        --------cool--------              ultimate  18
24               -------_-----             AskReddit   1
25               -------_-----  Competitiveoverwatch  12
26               -------_-----            Futurology   2
27               -------_-----       GlobalOffensive  26
28               -------_-----                 Jokes   1
29               -------_-----          OurPresident   2
30               -------_-----             Overwatch  15
31               -------_-----   OverwatchUniversity   3
32               -------_-----                 funny   2
33               -------_-----                gaming   1
34               -------_-----               

In [13]:
N_CLUSTERS = 1000
N_COMPONENTS = 300

print(len(users_id))
print(user_item_matrix.shape)

# On convertit la matrice dans un format adapté
X = csr_matrix(user_item_matrix).transpose()
print("Transposed")

# On effectue une réduction de dimension pour éviter la malédiction de la dimension
tsvd = TruncatedSVD(n_components = N_COMPONENTS)
X = tsvd.fit_transform(X)
print("Truncated")

# On effectue le clustering
db = KMeans(n_clusters=N_CLUSTERS).fit_predict(X)
print("Finished")

695567
(695616, 32607)
Transposed
Truncated
Finished


In [14]:
# Affichage des résultats
for n in range(N_CLUSTERS):
    l = []
    for subreddit in subreddits_id:
        cluster = db[subreddits_id[subreddit]]
        if cluster == n:
            l.append(subreddit)
    print("Cluster n°{}, size: {}".format(n, len(l)))
    print(l)
    print("\n")

Cluster n°0, size: 43
['kittensgame', 'BuffaloWizards', 'deeeepio', 'ralphthemoviemaker', 'NightMusic', 'Persona5Countdown', 'sigurros', 'BringMeTheHorizon', 'combinedarms', 'fcpx', 'Trimps', 'theavalanches', 'Homeshake', 'minecraftbingo', 'MuscleConfusion', 'vanderpumprules', 'TheOverload', 'askashittyphilosopher', 'madeon', 'armadev', 'Grimes', 'losgrowlers', 'Waifu', 'TySegall', 'imperialruler', 'alterbridge', 'stonehearth', 'language', 'ModelWHPress', 'formcheck', 'bayeuxtapestry', 'algeria', 'ImaginaryWTF', 'itrunsdoom', 'MGSVIDeathStranding', 'makemeaplaylist', 'PeanutButterGamer', 'PerfectPause', 'tha_pit', 'hayleywilliams', 'technicalminecraft', 'destinylegacy', 'musicanova']


Cluster n°1, size: 3
['OCD', 'canberra', 'tasmania']


Cluster n°2, size: 8
['SSBPM', 'CODModernWarfare', 'HunterXHunter', 'CallOfDuty', 'Cod4Remastered', 'dbxv', 'rutgers', 'TeamFourStar']


Cluster n°3, size: 7
['PSNFriends', 'gravityrush', 'thelastofus', 'PlayStationPlus', 'assassinscreed', 'kindafunn

Cluster n°42, size: 11
['Luna_Lovewell', 'TheCryopodToHell', 'YouEnterADungeon', 'Pubby88', 'nickofnight', 'ghost_write_the_whip', 'promptoftheday', 'DailyChat', 'thebookofrad', 'ZigZagStories', 'Inooxwritings']


Cluster n°43, size: 10
['whitepeoplefacebook', 'transformation', 'InfamousSecondRP', 'WorldChallenges', 'Worldprompts', 'FreeAtheism', 'chuggaaconroy', 'PastorArrested', 'neography', 'bluekrill_']


Cluster n°44, size: 7
['oculus', 'Vive', 'RecRoom', 'H3VR', 'GearVR', 'virtualreality', 'OnwardVR']


Cluster n°45, size: 8
['InstrumentPorn', 'EASportsUFC', 'Kickboxing', 'heavyvinyl', 'MMAFantasy', 'mmapredictions', 'mmamemes', 'mmadadjokes']


Cluster n°46, size: 7
['USNEWS', 'Nationalism', 'The_Sweden', 'European_New_Right', 'IdentityIrelandForum', 'IrishNationalism', 'euromigration']


Cluster n°47, size: 14
['toppkommentatorene', 'Maps', 'swedishproblems', 'Allsvenskan', 'Norse', 'swedents', 'SwedditUniversalis', 'iksdagen', 'unket', 'dialekter', 'Asksweddit', 'pinsamt', 'Ti

Cluster n°78, size: 1
['NGolo_Kante']


Cluster n°79, size: 78
['Guadalajara', 'NCLEX', 'GHOSTEMANE', 'Urology', 'braincancer', 'LONESTAR', 'pharmaceuticals', 'SFEvents', 'MoonriseMusicFestival', 'lojban', 'TheBukuProject', 'H1Z1clans', 'Nicaragua', 'Sherbrooke', 'paradisofestival', 'StopSpeeding', 'herbalism', 'hulaween', 'dubuque', 'sexuality', 'LoMejorDeNetflixMX', 'whatthefestival', 'attorneys', 'Ceretropic', 'chivas', 'haxball', 'DPH', 'Tobacco', 'EilanCraga', 'recovery', 'PsychedSubstance', 'terencemckenna', 'RockBalancing', 'Clonazolam', 'jacksonms', 'ConsumerAdvice', 'entheogens', 'tryptonaut', 'benzorecovery', 'ensenada', 'nsfw_toned', 'bassline', 'phishcirclejerk', 'mentalillness', 'olling', 'musicwallpapers', 'kidney', 'COTreeGifting', 'Halberstadt', 'neuroproducers', 'infectiousdisease', 'Peptides', 'ps4friends', 'Sauna', 'jeopardyhot', 'AncientWorld', 'PsychedelicStudies', 'bikedc', 'JFK', 'introvents', 'Pattaya', 'RandomTestingGround', 'Michoacan', 'nitrogensports', 'Herb

Cluster n°91, size: 9
['Cascadia', 'CBD', 'productivity', 'NavyBlazer', 'singularity', 'TeenFA', 'eldertrees', 'malelifestyle', 'Leathercraft']


Cluster n°92, size: 7
['BeaconAcademyNSFW', 'Skullgirls', 'hextcg', 'l5r', 'makeyourchoice', 'erfworld', 'monsterdeconstruction']


Cluster n°93, size: 1
['Existentialism']


Cluster n°94, size: 4
['Roast_Me_', 'oastme', 'roastme_unlimited', 'PhoenixTheBand']


Cluster n°95, size: 45
['tdi', 'Redboid', 'Robocraft', 'Jobs4Bitcoins', 'wichita', 'fitnesscirclejerk', 'ZenHabits', 'Incel', 'vagabond', 'lawofattraction', 'lacrosse', 'Tomorrowland', 'lasercutting', 'mash', 'ENFP', 'PuertoRico', 'volleyball', 'diydrones', 'appliancerepair', 'SpiceandWolf', 'Pomeranians', 'IceFishing', 'mobilerepair', 'Clemson', 'AskMechanics', 'cannabiscultivation', 'dawnofwar', 'XVcrosstrek', 'londoncycling', 'UCDavis', 'Athens', 'burbank', 'FinalFantasyTCG', 'firefighters', 'Tulpas', 'maximumfun', 'exalted', 'Motorrad', 'DanceDanceRevolution', 'OfficeDepot', 'FZ09'

Cluster n°121, size: 27
['pathofexiledev', 'vgstreams', 'brasil_drama', 'riodejaneiro', 'Skyforge', 'ChroniclesOfElyria', 'PoEBuilds', 'AgeOfConanUC', 'theislandsofnyne', 'AshesofCreation', 'New_World_MMO', 'brdev', 'SCUMgame', 'Wolcen', 'Chronicon', 'brasilia', 'Soberano', 'memesinspiradores', 'galacticjunkleague', 'ConanExilesAdmins', 'warface', 'RidersofIcarus', 'direitosdoshomens', 'conanreapers', 'palmeiras', 'AcolytesCimmeria', 'Ginfo']


Cluster n°122, size: 27
['amumumains', 'Launchpad', 'MHOnline', 'reksaimains', 'LOLDyrus', 'Saekano', 'vandiril', 'sejuanimains', 'LoLCommunity', 'Shurima', 'houseparty5v5', 'DarknessKS', 'SuperstarJYPNation', 'Asuka', 'whoreanddogscj', 'Pingponggirls', 'yunyun', 'TouhouArt', 'ahritime', 'ultimatedetective', 'lookingforheroes', 'plamemo', 'ADCMains', 'Lolboosting', 'LOLSUMO', 'TheGrandWaifu', 'loldads']


Cluster n°123, size: 134
['AnimalsBeingConfused', 'letterswap', 'Kyoto', 'Namibia', 'Robocrafters', 'JohnCena', 'FoundFace', 'SuperAthleteGifs

...

Cluster n°889, size: 19
['weightroom', 'pharmacy', 'pacers', 'astoria', 'milwaukee', 'declutter', 'peacecorps', 'Blackfellas', 'amiibo', 'immigration', 'Virginia', 'AmItheAsshole', 'labrats', 'pregnant', 'Assistance', 'RedPillWomen', 'FemmeThoughts', 'JuneBumpers2017', 'CautiousBB']


Cluster n°890, size: 166
['unsimulated', 'CuteModeSlutMode', 'LittleCaprice', 'newreddits_nsfw', 'CelebrityAnus', 'Whoregasm', 'MalenaMorgan', 'Kazaaporn', 'Porn_Discussion', 'PrismaPornArt', 'DakotaSkye', 'felching', 'Pornhub', 'VaginalAnalOral', 'MeganFox', 'maseratixxx', 'BigBrotherNSFW', 'CageTheElephant', 'JaydenJaymes', 'TaylorAlesia', 'JerkingHimOntoHer', 'BrandiLove', 'JailyneOjedaOchoa', 'IncestComics', 'cumbtwntits', 'Sexyvelma', 'dillion_harper', 'RachelStarr', 'CelebrityTributes', 'SSSniperWolfLust', 'arielrebel', 'BrooklynChase', 'JennaJade', 'NiykeeHeaton', 'squirting_gifs', 'peachfuzz', 'SupeApp', 'AnarchyChess', 'CelestiaVega', 'Pulldown', 'Asian_Bondage', 'ChineseHotties', 'FoxyDi', 'QVCT

Cluster n°903, size: 8
['NSFWFunny', 'MMA', 'watchpeopledie', 'gentlemanboners', 'WatchItForThePlot', 'cosplaygirls', 'trashyboners', 'PrettyGirls']


Cluster n°904, size: 2
['eastside', 'portlandstate']


Cluster n°905, size: 10
['powerrangers', 'transformers', 'Heavymind', 'xmen', 'HappyTrees', 'learnart', 'Illustration', 'Watercolor', 'sketches', 'fanart']


Cluster n°906, size: 63
['NintendoKaren', 'kati3kat', 'MirandaCosgrove', 'Zoo', 'ElizaTaylor', 'camilamendes', 'lilireinhart', 'BrieLarson', 'TurtleBeachHeadsets', 'MomInstincts', 'ninteendo', 'AnneHathaway', 'Incorporated', 'JennaLouiseColeman', 'CasualTodayILearned', 'abolish', 'GillianJacobs', 'NearlyChristianMusic', 'WomenWithWatches', 'faithschroder', 'SmashBrosSwitch', 'CamrenBicondova', 'PintmanPosting', 'PerthSwitch', 'DCAndMarvelwomen', 'CelebsNL', 'CamilaMorrone', 'AvaSambora', 'Anya_TaylorJoy', 'nyctohylophobia', 'hotukdealsuk', 'NintedoSwitch', 'KlaraCastanho', 'AnalCowgirl', 'GameDevTycoon', 'annasophiarobb', 'Sofia

Cluster n°935, size: 114
['UsedCars', 'singlespeedcycling', 'shittyengineering', 'Traderfeedback', 'BarnFinds', 'Hotrod', 'FordRaptor', 'cranes', 'OpenEmu', 'uglybabies', 'LiftingRoutines', 'LiftingVideos', '14ers', 'fiero', 'bathbeer', 'SignatureRequests', 'worms', 'bronco', 'Tavor', 'westcoasteagles', 'infiniteworldproblems', 'Graymuzzles', 'privatestudyrooms', 'rfactor2', 'RunnerHub', 'geophysics', 'Hatchback', 'AutoPaint', 'UAVmapping', 'peugeot', 'austinjobs', 'JDM_', 'Chinavisa', 'Salomondrin', 'gifrequests', 'FantasyNRL', 'car', 'Fishing_Gear', 'panamacity', 'motograndprix', 'mclaren', 'overland', 'gender', 'ShitCosmoSays', 'NFL2go', 'askmusicians', 'Rotaries', 'LandroverDefender', 'skilledtrades', 'IS300', 'FoodDev', 'grammarfail', 'AwfulCommercials', 'viper', 'ShinJimin', 'RangeRover', 'e30classifieds', 'mkbhd', 'UoNau', 'WorldRallyCross', 'JeepLiberty', 'PokemonGoAdelaide', 'utvoffroad', 'WeHaveSeenTheButthole', 'Forts', 'vintagef1', '4Runner_1stGen', 'Nurnberg', 'Artemis', '

Cluster n°962, size: 32
['goldenknights', 'ahl', 'hockeyFandom', 'FictionBrawl', 'sebastianaho', 'RenttheJoe', 'poutine', 'hockeycirclejerk', 'KontinentalHL', 'EANHLfranchise', 'SanJoseBarracuda', 'hockeyrefs', 'hockeycards', 'hawksmarketplace', 'EHM', 'nhl_games', 'spokanechiefs', 'OldTimeHockey', 'MichelleTrachtenberg', 'TheLastKingdom', 'ShotGlassBets', 'JayandDan', 'hockeycardswap', 'PuckSoup', 'mvsw', 'HGD', 'norfolkadmirals', 'BeautyQueens', 'gifsfromthegame', 'MilwaukeeAdmirals', 'ECHL', 'BUHockey']


Cluster n°963, size: 28
['wow', 'GlobalOffensive', 'OverwatchUniversity', 'playrust', 'buildapcsales', 'heroesofthestorm', 'hearthstone', 'battlestations', 'DestinyTheGame', '2007scape', 'kotk', 'thewalkingdead', 'Diablo', 'Suomi', 'customhearthstone', 'Infinitewarfare', 'pathofexile', 'elderscrollsonline', 'photography', 'AyyMD', 'LivestreamFail', 'ConanExiles', 'Smite', 'exmormon', 'playark', 'blackops3', 'Twitch', 'CODZombies']


Cluster n°964, size: 24
['paulthomasanderson', 'S

Cluster n°994, size: 28
['Battlefield', 'battlefield_one', 'HiTMAN', 'masseffect', 'residentevil', 'thedivision', 'playstation', 'GrandTheftAutoV', 'titanfall', 'GearsOfWar', 'halo', 'Rainbow6', 'fo4', 'Wildlands', 'FFXV', 'GhostRecon', 'NoMansSkyTheGame', 'gtaonline', 'ForHonorVikings', 'forhonorknights', 'witcher', 'airsoft', 'battlefield_4', 'watch_dogs', 'gwent', 'shittyrainbow6', 'metalgearsolid', 'GTAV']


Cluster n°995, size: 26
['MakeUpFetish', 'WedgieGirls', 'AvaAddams', 'Guiltygear', 'edging', 'killerinstinct', 'HugeDildos', 'Nina_Agdal', 'blacktears', 'Gloryholes', 'BrittneyWhite', 'titplaysex', 'blowjob_gifs', 'CuteLittleButts', 'jennettemccurdy', 'NSFW_ASS', 'leahgotti', 'Best_NSFW_Content', 'girlsflashing', 'NSFW_hardcore', 'fishnets', 'SaggyTits', 'smokingfetish', 'BarelyContained', 'StarletGIFS', 'Pornstars_NSFW']


Cluster n°996, size: 144
['thick_hentai', 'braceface', 'PussyFlashing', 'WendyFiore', 'leannecrow', 'ChantelZales', 'MadisonGordon', 'LexiBelle', 'PussyStub

# Comparaison des clusters obtenus et conclusion

Les deux approches nous permettent d'obtenir des clusters qui semblent cohérents à première vue (il est bien entendu impossible de tout analyser humainement).

Néanmoins, il semble que l'apporche exploitant les utilisateurs soit plus aléatoire. En effet, les clusters obtenus peuvent être de qualité variable selon l'exécution (l'algorithme KMeans initialisant ses centroïdes de façon aléatoire, les clusters sont eux-mêmes aléatoires). La première approche semble être moins sensible à ce côté aléatoire.

Par ailleurs, les deux approches font apparaître quelques différences de clusters. Typiquement, l'approche par vocabulaire fait apparaître un cluster contenant des proposition d'emplois dans plusieurs villes du monde (cluster n°21 de la première approche). Ce type de cluster ne peut pas apparaître dans la seconde approche car un utilisateur intéressé par un travail à Washington ne l'est sûrement pas par un travail à Londres. Mais le vocabulaire utilisé dans ces subreddits est proche, d'où le résultat dans la première approche.

Nous nous excusons une nouvelle fois pour les subreddits peu recommandables qui sont apparus dans certains clusters.